## 训练和验证数据集
list(np.random.permutation(10))

[5, 1, 7, 6, 8, 9, 4, 0, 2, 3]

In [28]:
def split_indices(n, val_pct):
    # Determine size of validation set
    n_val = int(val_pct*n)
    # Create random permutation of 0 to n-1
    idxs = np.random.permutation(n)
    # Pick first n_val indices for validation set
    return idxs[n_val:], idxs[:n_val]

In [29]:
train_indices, val_indices = split_indices(len(dataset), val_pct=0.2)

In [30]:
print(len(train_indices), len(val_indices))
print('Sample val indices: ', val_indices[:20])

48000 12000
Sample val indices:  [38545 55218 33568 42571 54023 19408 17517 25742  8024 22876 39746  8917
  3227 32420 19246 42479 18072 42284 25179 32022]


In [31]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [32]:
batch_size=100

# Training sampler and data loader
train_sampler = SubsetRandomSampler(train_indices)
train_loader = DataLoader(dataset, 
                          batch_size, 
                          sampler=train_sampler)

# Validation sampler and data loader
val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset,
                        batch_size, 
                        sampler=val_sampler)

In [33]:
import torch.nn as nn

input_size = 28*28
num_classes = 10

# Logistic regression model
model = nn.Linear(input_size, num_classes)

In [34]:
print(model.weight.shape)
model.weight

torch.Size([10, 784])


Parameter containing:
tensor([[-0.0019, -0.0324,  0.0198,  ..., -0.0190, -0.0103,  0.0137],
        [-0.0245,  0.0036,  0.0236,  ..., -0.0158, -0.0042,  0.0167],
        [-0.0060, -0.0021, -0.0120,  ..., -0.0095, -0.0310, -0.0200],
        ...,
        [-0.0036,  0.0232,  0.0003,  ...,  0.0195, -0.0294, -0.0024],
        [ 0.0028, -0.0038, -0.0091,  ..., -0.0139,  0.0003, -0.0194],
        [-0.0294, -0.0256, -0.0281,  ...,  0.0160, -0.0293, -0.0307]],
       requires_grad=True)

In [35]:
print(model.bias.shape)
model.bias

torch.Size([10])


Parameter containing:
tensor([-0.0158, -0.0266, -0.0265, -0.0178,  0.0090,  0.0144, -0.0156, -0.0155,
         0.0029,  0.0139], requires_grad=True)

In [36]:
for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [37]:
class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 784)
        out = self.linear(xb)
        return out
    
model = MnistModel()

In [38]:
print(model.linear.weight.shape, model.linear.bias.shape)
list(model.parameters())

torch.Size([10, 784]) torch.Size([10])


[Parameter containing:
 tensor([[-0.0038,  0.0035, -0.0266,  ..., -0.0160, -0.0242,  0.0257],
         [ 0.0193, -0.0113,  0.0228,  ..., -0.0179, -0.0321, -0.0213],
         [-0.0273,  0.0317, -0.0345,  ...,  0.0184, -0.0132, -0.0104],
         ...,
         [-0.0344, -0.0214,  0.0192,  ..., -0.0070, -0.0201,  0.0252],
         [ 0.0076, -0.0294,  0.0294,  ...,  0.0258, -0.0271, -0.0326],
         [ 0.0219,  0.0321,  0.0143,  ..., -0.0108, -0.0030,  0.0313]],
        requires_grad=True), Parameter containing:
 tensor([ 0.0030, -0.0153, -0.0232,  0.0141,  0.0329, -0.0062, -0.0014,  0.0279,
          0.0232,  0.0173], requires_grad=True)]

In [39]:
for images, labels in train_loader:
    outputs = model(images)
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [40]:

import torch.nn.functional as F

In [41]:
# Apply softmax for each output row
probs = F.softmax(outputs, dim=1)

# Look at sample probabilities
print("Sample probabilities:\n", probs[:2].data)

# Add up the probabilities of an output row
print("Sum: ", torch.sum(probs[0]).item())

NameError: name 'outputs' is not defined

In [42]:

max_probs, preds = torch.max(probs, dim=1)
print(preds)

NameError: name 'probs' is not defined

In [43]:
labels

NameError: name 'labels' is not defined

## 评测和损失函数

In [ ]:
def accuracy(l1, l2):
    return torch.sum(l1 == l2).item() / len(l1)
accuracy(preds, labels)

In [ ]:
loss_fn = F.cross_entropy

In [ ]:
# Loss for current batch of data
loss = loss_fn(outputs, labels)
print(loss)

## 优化器

In [ ]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 训练模型

In [44]:
def loss_batch(model, loss_func, xb, yb, opt=None, metric=None):
    # Calculate loss
    preds = model(xb)
    loss = loss_func(preds, yb)
                     
    if opt is not None:
        # Compute gradients
        loss.backward()
        # Update parameters             
        opt.step()
        # Reset gradients
        opt.zero_grad()
    
    metric_result = None
    if metric is not None:
        # Compute the metric
        metric_result = metric(preds, yb)
    
    return loss.item(), len(xb), metric_result

In [45]:
def evaluate(model, loss_fn, valid_dl, metric=None):
    with torch.no_grad():
        # Pass each batch through the model
        results = [loss_batch(model, loss_fn, xb, yb, metric=metric)
                   for xb,yb in valid_dl]
        # Separate losses, counts and metrics
        losses, nums, metrics = zip(*results)
        # Total size of the dataset
        total = np.sum(nums)
        # Avg. loss across batches 
        avg_loss = np.sum(np.multiply(losses, nums)) / total
        avg_metric = None
        if metric is not None:
            # Avg. of metric across batches
            avg_metric = np.sum(np.multiply(metrics, nums)) / total
    return avg_loss, total, avg_metric


In [46]:

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(preds)

In [ ]:
val_loss, total, val_acc = evaluate(model, loss_fn, val_loader, metric=accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}'.format(val_loss, val_acc))

In [ ]:

def fit(epochs, model, loss_fn, opt, train_dl, valid_dl, metric=None):
    for epoch in range(epochs):
        # Training
        for xb,yb in train_dl:
            loss,_,_ = loss_batch(model, loss_fn, xb, yb, opt)

        # Evaluation
        result = evaluate(model, loss_fn, valid_dl, metric)
        val_loss, total, val_metric = result
        
        # Print progress
        if metric is None:
            print('Epoch [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, epochs, val_loss))
        else:
            print('Epoch [{}/{}], Loss: {:.4f}, {}: {:.4f}'
                  .format(epoch+1, epochs, val_loss, metric.__name__, val_metric))

In [ ]:
# Redifine model and optimizer
model = MnistModel()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:

fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:

fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
fit(5, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

In [ ]:
# Replace these values with your results
accuracies = [0.1076, 0.6486, 0.7362, 0.7737, 0.7925, 0.8069, 
              0.8165, 0.8227, 0.8269, 0.8325, 0.8367,
              0.8399, 0.8438, 0.8463, 0.8482, 0.8512,
              0.8529, 0.8552, 0.8563, 0.8587, 0.8606]
plt.plot(accuracies, '-x')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('Accuracy vs. No. of epochs');

## 测试验证集

In [47]:
# Define test dataset
test_dataset = MNIST(root='data/', 
                     train=False,
                     transform=transforms.ToTensor())

NameError: name 'transforms' is not defined

In [ ]:
img, label = test_dataset[0]
plt.imshow(img[0], cmap='gray')
print('Shape:', img.shape)
print('Label:', label)